In [0]:
import sys
sys.path.append("/Workspace/Users/joaocardoso.dataengineer@outlook.com/databricks_dbt/PROJECTS")

from utils.BaseIngestion import BaseIngestion

In [0]:
class BronzeIngestion(BaseIngestion):
    def __init__(self):
        super().__init__(layer="bronze")
        self.file_format = "csv"
        self.schema_evolution = "rescue"

    def ingest_data(self):
        df = spark.readStream.format("cloudFiles") \
            .option("cloudFiles.format", self.file_format) \
            .option("cloudFiles.schemaLocation", self.schema_location) \
            .option("cloudFiles.schemaEvolutionMode", self.schema_evolution) \
            .load(self.base_path)
        return df

    def load_data(self, df):
        df.writeStream \
            .format("delta") \
            .outputMode("append") \
            .trigger(once=True) \
            .option("checkpointLocation", self.schema_location) \
            .option("path", self.output_path) \
            .start()

    def main(self):
        df = self.ingest_data()
        self.load_data(df)


In [0]:
obj = BronzeIngestion()
obj.main()

In [0]:
%sql DESCRIBE HISTORY delta.`/Volumes/one_lake_dev/bronze/landzone/Flights/fct/Bookings/data`

In [0]:
display(dbutils.fs.ls("/Volumes/one_lake_dev/bronze/landzone/Flights/fct/Bookings/checkpoint/_schemas/0"))